In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE106800"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE106800"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE106800.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE106800.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE106800.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Circadian misalignment induces fatty acid metabolism gene profiles and induces insulin resistance in human skeletal muscle."
!Series_summary	"Circadian misalignment, such as in shift work, has been associated with obesity and type 2 diabetes, however, direct effects of circadian misalignment on skeletal muscle insulin sensitivity and muscle molecular circadian clock have never been investigated in humans. Here we investigated insulin sensitivity and muscle metabolism in fourteen healthy young lean men (age 22.4 ± 2.8 years; BMI 22.3 ± 2.1 kg/m2 [mean ± SD]) after a 3-day control protocol and a 3.5-day misalignment protocol induced by a 12-h rapid shift of the behavioral cycle. We show that circadian misalignment results in a significant decrease in peripheral insulin sensitivity due to a reduced skeletal muscle non-oxidative glucose disposal (Rate of disappearance: 23.7 ± 2.4 vs. 18.4 ± 1.4 mg/kg/min; control vs. misalignment; p=0.024). Fasting gl

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
is_gene_available = True  # Based on background information, this is microarray data from skeletal muscle biopsies

# 2.1 Identify rows for variables
trait_row = 3  # height (m) is at key 3
age_row = 2    # age (yrs) is at key 2
gender_row = 0  # gender is at key 0

# 2.2 Define conversion functions for each variable
def convert_trait(value):
    """Convert height value to float (continuous)"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract value after colon
    parts = value.split(': ')
    if len(parts) < 2:
        return None
    
    height_str = parts[1].strip()
    try:
        # Convert to meters as float
        return float(height_str)
    except ValueError:
        return None

def convert_age(value):
    """Convert age value to integer (continuous)"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract value after colon
    parts = value.split(': ')
    if len(parts) < 2:
        return None
    
    age_str = parts[1].strip()
    try:
        # Convert to years as integer
        return int(age_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract value after colon
    parts = value.split(': ')
    if len(parts) < 2:
        return None
    
    gender = parts[1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    else:
        return None

# 3. Save metadata for initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available and we can access the sample characteristics
if trait_row is not None:
    # In this step, we're just verifying that trait data is available
    # We'll process the actual data in a later step after we have access to the required files
    print(f"Trait data is available at row {trait_row} (height in meters)")
    if age_row is not None:
        print(f"Age data is available at row {age_row}")
    if gender_row is not None:
        print(f"Gender data is available at row {gender_row}")
    
    print(f"Initial validation complete. This dataset passes the initial filtering criteria.")


Trait data is available at row 3 (height in meters)
Age data is available at row 2
Gender data is available at row 0
Initial validation complete. This dataset passes the initial filtering criteria.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 74
Header line: "ID_REF"	"GSM2850460"	"GSM2850461"	"GSM2850462"	"GSM2850463"	"GSM2850464"	"GSM2850465"	"GSM2850466"	"GSM2850467"	"GSM2850468"	"GSM2850469"	"GSM2850470"	"GSM2850471"	"GSM2850472"	"GSM2850473"	"GSM2850474"	"GSM2850475"	"GSM2850476"	"GSM2850477"	"GSM2850478"	"GSM2850479"	"GSM2850480"	"GSM2850481"	"GSM2850482"	"GSM2850483"	"GSM2850484"	"GSM2850485"	"GSM2850486"	"GSM2850487"	"GSM2850488"	"GSM2850489"	"GSM2850490"	"GSM2850491"	"GSM2850492"	"GSM2850493"	"GSM2850494"	"GSM2850495"	"GSM2850496"	"GSM2850497"	"GSM2850498"	"GSM2850499"	"GSM2850500"	"GSM2850501"	"GSM2850502"	"GSM2850503"	"GSM2850504"	"GSM2850505"	"GSM2850506"
First data line: 16650001	2.440491762	1.309586553	1.845734791	1.948754211	0.533478717	1.828953915	1.098916463	1.570142428	1.193317567	1.66634646	3.03320333	1.216260139	1.61344107	1.441936181	1.222761955	1.117259019	1.3918389	1.3962103	1.457018968	1.920330181	1.355983375	1.965290311	2.183128158	0.614994555	1.466733313	1.080307749	1.34161

Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyze gene identifiers
# The identifiers appear to be probe IDs or numeric identifiers (like 16650001, 16650003, etc.)
# These are not standard human gene symbols, which would typically be like BRCA1, TP53, etc.
# Therefore, we need to map these identifiers to proper gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE106800
Line 6: !Series_title = Circadian misalignment induces fatty acid metabolism gene profiles and induces insulin resistance in human skeletal muscle.
Line 7: !Series_geo_accession = GSE106800
Line 8: !Series_status = Public on Aug 10 2018
Line 9: !Series_submission_date = Nov 13 2017
Line 10: !Series_last_update_date = Aug 12 2018
Line 11: !Series_pubmed_id = 29987027
Line 12: !Series_summary = Circadian misalignment, such as in shift work, has been associated with obesity and type 2 diabetes, however, direct effects of circadian misalignment on skeletal muscle insulin sensitivity and muscle molecular circadian clock have never been investigated in humans. Here we investigated insulin sensitivity an


Gene annotation preview:
{'ID': [16657436, 16657440, 16657445, 16657447, 16657450], 'probeset_id': [16657436, 16657440, 16657445, 16657447, 16657450], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25, 28, 8, 13, 36], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene annotation for probe IDs and gene symbols
# Looking at the data structure:
# - 'ID' column contains numeric identifiers (16657436, etc) which match the gene expression data's index (16650001, etc)
# - 'gene_assignment' column contains information about genes with gene symbols (like DDX11L1, MIR1302-11, OR4F5)

# 2. Extract gene mapping using ID and gene_assignment columns
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# Let's examine a few rows of the mapping data to verify it's correct
print("Gene mapping preview:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Verify the structure of the resulting gene expression data
print("\nGene expression data preview:")
print(f"Number of genes: {len(gene_data)}")
print(f"Number of samples: {len(gene_data.columns)}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Save gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping preview:
         ID                                               Gene
0  16657436  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  16657440  ENST00000473358 // MIR1302-11 // microRNA 1302...
2  16657445  NM_001005484 // OR4F5 // olfactory receptor, f...
3  16657447                                                ---
4  16657450  AK302511 // LOC100132062 // uncharacterized LO...



Gene expression data preview:
Number of genes: 81076
Number of samples: 47
First few gene symbols:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1'], dtype='object', name='Gene')



Gene expression data saved to ../../output/preprocess/Height/gene_data/GSE106800.csv


### Step 7: Data Normalization and Linking

In [8]:
import numpy as np
import os
import gzip

# 1. Extract gene expression data using the alternative approach that worked in Step 3
with gzip.open(matrix_file, 'rt') as file:
    # Find the start of the data
    for line in file:
        if '!series_matrix_table_begin' in line:
            break
            
    # Read the headers and data
    gene_data = pd.read_csv(file, sep='\t', index_col=0)

# Check if we have gene data before proceeding
if gene_data.empty:
    print("No gene expression data found in the matrix file.")
    is_gene_available = False
else:
    is_gene_available = True
    print(f"Gene data shape before normalization: {gene_data.shape}")

    # Normalize gene symbols using the NCBI Gene database information
    try:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
        
        # Save the normalized gene data to the output file
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        normalized_gene_data.to_csv(out_gene_data_file)
        print(f"Normalized gene data saved to {out_gene_data_file}")
    except Exception as e:
        print(f"Error normalizing gene data: {e}")
        is_gene_available = False
        normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Link clinical and genetic data
# In Step 2, we determined that Height data is not available in this dataset (trait_row = None)
# Create a minimal clinical dataframe with the trait column (containing NaNs)
if is_gene_available:
    sample_ids = gene_data.columns
    minimal_clinical_df = pd.DataFrame(index=sample_ids)
    minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values

    # If we have age and gender data from Step 2, add those columns
    if age_row is not None:
        minimal_clinical_df['Age'] = get_feature_data(clinical_data, age_row, 'Age', convert_age).iloc[0]

    if gender_row is not None:
        minimal_clinical_df['Gender'] = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender).iloc[0]

    minimal_clinical_df.index.name = 'Sample'

    # Save this minimal clinical data for reference
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    minimal_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

    # Create a linked dataset 
    if is_gene_available and normalized_gene_data is not None:
        linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
        linked_data.index.name = 'Sample'
        print(f"Linked data shape: {linked_data.shape}")
    else:
        linked_data = minimal_clinical_df
        print("No gene data to link with clinical data.")
else:
    # Create a minimal dataframe with just the trait for the validation step
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No gene data available, creating minimal dataframe for validation.")

# 4 & 5. Validate and save cohort information
# Since trait_row was None in Step 2, we know Height data is not available
is_trait_available = False  # Height data is not available

note = "Dataset contains gene expression data but no Height measurements. This dataset is not usable for studying Height associations."

# For datasets without trait data, we set is_biased to False
# This indicates the dataset is not usable due to missing trait data, not due to bias
is_biased = False

# Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Since there is no trait data, the dataset is not usable for our association study
# So we should not save it to out_data_file
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset does not contain Height data and cannot be used for association studies.")

Gene data shape before normalization: (53618, 47)
Gene data shape after normalization: (0, 47)
Normalized gene data saved to ../../output/preprocess/Height/gene_data/GSE106800.csv
Clinical data saved to ../../output/preprocess/Height/clinical_data/GSE106800.csv
Linked data shape: (47, 3)
Abnormality detected in the cohort: GSE106800. Preprocessing failed.
Dataset usability: False
Dataset does not contain Height data and cannot be used for association studies.


/tmp/ipykernel_38861/939407384.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_data = pd.read_csv(file, sep='\t', index_col=0)
